In [ ]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(ggraph)
library(igraph)

In [ ]:
df <- lapply(snakemake@input$info, function(x){
    return(fread(x))
})
zos <- unlist(lapply(df,nrow))
df2 <- bind_rows(df[zos!=0]) #%>%
    #group_by(sbin, qbin) #%>%
    # summarise(N=n()) %>%
    # ungroup() %>%
    # group_by(donor, reciever) %>%
    # summarise(N=max(N), bins=paste(unlist(unique(bin)),collapse=",")) %>%
    # ungroup() %>%
    # filter(N > 2, donor != reciever)
fwrite(x = df2,file = snakemake@output$csv)

In [ ]:
df3 <- df2 %>%
    group_by(sbin, qbin) %>%
    summarise(N=n()) %>%
    ungroup() %>%
    rename(donor = sbin, reciever = qbin)
df3

In [ ]:
graph_data <- df3 %>%
  dplyr::select(donor, reciever, N) %>%  # Ensure you're only taking necessary columns
  graph_from_data_frame(directed = TRUE) # Create a directed graph


In [ ]:
library(ggraph)
library(igraph)

# Adjusting the options for better SVG output in Jupyter
options(repr.plot.width = 12, repr.plot.height = 5, jupyter.plot_mimetypes = "image/svg+xml")

# Re-evaluate the use of layout if x and y coordinates should be preserved 
p1 <- ggraph(graph_data, layout = 'linear') + 
  geom_edge_arc(aes(width = N),  # Apply curvature outside of aes()
                arrow = arrow(length = unit(3, 'mm'), type = 'closed', ends = 'last',), color = "red",  alpha = 0.5) +

  geom_edge_arc(aes(width = N),  # Apply curvature outside of aes()
                , color = "grey",  alpha = 0.5) +
  geom_node_text(aes(label = name), vjust = 0, hjust = 0.1, 
                 color = "black", check_overlap = F, size = 5,  repel = TRUE, force=200) +
  geom_node_point(color = "darkblue", size = 2, alpha = 0.5) +

  theme(legend.position = "bottom",
        plot.background = element_rect(fill = "white", colour = NA),  
        panel.background = element_rect(fill = "white", colour = NA),
  legend.text = element_text(size = 12),  # Increase legend text size
        legend.title = element_text(size = 14),  # Increase legend title siz       
       ) #+
#coord_cartesian(xlim=c(-1.5,1.5), ylim=c(-1.1,1.2))
p1

In [ ]:
library(ggraph)
library(igraph)

# Adjusting the options for better SVG output in Jupyter
options(repr.plot.width = 12, repr.plot.height = 5, jupyter.plot_mimetypes = "image/svg+xml")

# Assuming graph_data is already prepared as an igraph object
# Example to ensure names are set as vertex attributes:
# V(graph_data)$name <- c("Node1", "Node2", "Node3", ...)

# Set up the plot
p1 <- ggraph(graph_data, layout = 'linear') + 
  geom_edge_arc(aes(width = N, color = factor(to)),alpha = 0.5,  # Color by target node 'to'
                arrow = arrow(length = unit(3, 'mm'), type = 'closed', ends = 'last')) +
  geom_node_text(aes(label = name), vjust = 0, hjust = 0.1, 
                 color = "black", check_overlap = FALSE, size = 5, repel = TRUE, force = 200) +
  geom_node_point(color = "darkblue", size = 2, alpha = 0.5) +
  theme(legend.position = "bottom",
        plot.background = element_rect(fill = "white", colour = NA),  
        panel.background = element_rect(fill = "white", colour = NA),
        legend.text = element_text(size = 12),  # Increase legend text size
        legend.title = element_text(size = 14)) + # Increase legend title size +
    guides(color = "bin")


# Display the plot
print(p1)


In [ ]:
library(ggraph)
library(igraph)

# Adjusting the options for better SVG output in Jupyter
options(repr.plot.width = 12, repr.plot.height = 5, jupyter.plot_mimetypes = "image/svg+xml")

# Assuming graph_data is already prepared as an igraph object
# Example to ensure names are set as vertex attributes:
# V(graph_data)$name <- c("Node1", "Node2", "Node3", ...)

# Set up the plot
p1 <- ggraph(graph_data, layout = 'linear') + 
  geom_edge_arc(aes(width = N, color = factor(to)), alpha = 0.5,  # Color by target node 'to'
                arrow = arrow(length = unit(3, 'mm'), type = 'closed', ends = 'last')) +
  geom_node_text(aes(label = name), vjust = 0, hjust = 0.1, 
                 color = "black", check_overlap = FALSE, size = 5, repel = TRUE, force = 200) +
  geom_node_point(color = "darkblue", size = 2, alpha = 0.5) +
  theme(legend.position = "bottom",
        plot.background = element_rect(fill = "white", colour = NA),  
        panel.background = element_rect(fill = "white", colour = NA),
        legend.text = element_text(size = 12),  # Increase legend text size
        legend.title = element_text(size = 14)) + # Increase legend title size
    labs(color = "Receiver Node")  # Set custom title for the color legend

# Display the plot
print(p1)


In [ ]:
library(ggraph)
library(igraph)

# Adjust the options for better SVG output in Jupyter
options(repr.plot.width = 12, repr.plot.height = 5, jupyter.plot_mimetypes = "image/svg+xml")

# Assuming graph_data is already prepared as an igraph object
# Example to ensure that vertex and edge attributes are set up correctly:
# V(graph_data)$name <- c("Node1", "Node2", "Node3", ...)
# E(graph_data)$to <- V(graph_data)$name[head_of(edge_ids, n = ecount(graph_data))]  # Theoretically not needed, automatically managed by igraph

# Set up the plot
p1 <- ggraph(graph_data, layout = 'linear') + 
  geom_edge_arc(aes(width = N, color = factor(to)), alpha = 0.5,  # Color by the 'to' vertex attribute
                arrow = arrow(length = unit(3, 'mm'), type = 'closed', ends = 'last')) +
guides(color = "none") +

  geom_node_text(aes(label = name), vjust = 0, hjust = 0.1, 
                 color = "black", check_overlap = FALSE, size = 5, repel = TRUE, force = 200) +
  geom_node_point(color = "darkblue", size = 2, alpha = 0.5) +
  theme(legend.position = "bottom",
        plot.background = element_rect(fill = "white", colour = NA),  
        panel.background = element_rect(fill = "white", colour = NA),
        legend.text = element_text(size = 12),  # Increase legend text size
        legend.title = element_text(size = 14))   # Increase legend title size
# Display the plot
p1